<a href="https://colab.research.google.com/github/galik1999/Cloud_Branch/blob/main/Copy_of_CloudFlaskSite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyngrok==4.1.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15963 sha256=ffc195c50ebaafb6f31bd4d5ef2c8fe458f3a9d9b29e510ac38f8c32d70dccc8
  Stored in directory: /root/.cache/pip/wheels/4c/7c/4c/632fba2ea8e88d8890102eb07bc922e1ca8fa14db5902c91a8
Successfully built pyngrok


In [2]:
!pip install flask_ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install firebase

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
cd /content/drive/MyDrive/cloud/CloudPythonGameHtmlFiles

/content/drive/MyDrive/cloud/CloudPythonGameHtmlFiles


In [6]:
!ngrok authtoken "2Q6fmW7Ye5Ci4k3MuyLXHHa0Vub_54C7qnE6W5skndEQNJ5Mj"  

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template, url_for, request, redirect, abort, make_response, send_from_directory
import backend
import os 
app = Flask(__name__)
run_with_ngrok(app)
DBConnection = backend.Database('https://cloudclass-44ac5-default-rtdb.europe-west1.firebasedatabase.app/')


@app.route('/')
def index():
  r = make_response(render_template('index.html'))
  r.headers.set( "ngrok-skip-browser-warning", "69420")
  return r

@app.route('/<nickname>')
def home(nickname):
  db_users = DBConnection.get_data("Users")
  for user in db_users.values():
    if(nickname == user['nickname']):
      return render_template('index.html',nickname=nickname)
  return render_template('pageNotFound.html')

@app.route('/register')
def register():
  return render_template('register.html')

def score(element):
  return int(element["highscore"])

@app.route('/leaderboard')
def leaderboard():
  db_users = DBConnection.get_data("Users")
  db_list_users = [db_users[key] for key in db_users.keys()]
  db_list_users.sort(key=score, reverse=True)
  print(db_list_users)
  return render_template('leaderboard.html', db_list_users=db_list_users)

@app.route('/aboutUs')
def aboutUs():
  return render_template('AboutUs.html')

@app.route('/login')
def login():
  return render_template('login.html',flag=True)

@app.route('/login',methods=["POST"])
def loginAction(flag=True):
  db_users = DBConnection.get_data("Users")
  username = request.form.get('username')
  password = request.form.get('password')
  print(username,password)

  userData = {}
  for user in db_users.values():
    userData[user['username']] = user['password']
  
  print(userData)

  if(username not in userData.keys()):
    return render_template('login.html',flag=False)
  if(password not in userData.values()):
    return render_template('login.html',flag=False)
  if(userData[username] == password):
    # TODO: LOG IN
    print("SUCCESS")
    nickname = ""
    for user in db_users.values():
      if(user["username"] == username):
        nickname = user["nickname"]
        break
    
    return redirect(url_for('home', nickname=nickname))

  return render_template('login.html',flag=False)

@app.route('/register', methods=["POST"])
def registerpost():
  username = request.form.get('username')
  db_users = DBConnection.get_data("Users")
  print(db_users)
  is_duplicate = [key for key in db_users.keys() if db_users[key]["username"]==username]
  duplicated_users = list(is_duplicate)
  if(len(duplicated_users)):
    return abort(403)
  
  password = request.form.get('password')
  nickname = request.form.get('nickname')
  user_id = request.form.get('id')
  new_user = {
      "highscore":"0",
      "id":user_id,
      "nickname":nickname,
      "username":username,
      "password":password,
      "role": "player"
      }
  DBConnection.post_data("Users",new_user)
  return redirect(url_for('home', nickname=nickname))

@app.route('/manager/<qID>')
def getQuestion(qID):
  qArr = DBConnection.get_data("questions")
  qNumber = int(qID)
  print(qArr)
  return render_template('question.html',qArr=qArr,qNumber = qNumber)

@app.route('/manager/<qID>',methods=['POST'])
def operations(qID):
  if request.method == 'POST':
    if(request.form.get('delete') == 'Delete Question'):
      DBConnection.delete_data('questions',qID)
    elif(request.form.get('update') == 'Update Question'):
      print(request.form.get('update'))
      qArr = DBConnection.get_data('questions')
      key = len(qArr)
      # CRUD CREATE OPERATION
      description = request.form.get('desc')
      option1 = request.form.get('opt1')
      option2 = request.form.get('opt2')
      option3 = request.form.get('opt3')
      option4 = request.form.get('opt4')
      answer = request.form.get('corr')
      difficulty = request.form.get('diff')
      
      newQuestion = {
          'question': description,
          'option1': option1, 
          'option2': option2,
          'option3': option3,
          'option4': option4,
          'correct': answer,
          'questionlevel': difficulty    
      }
      DBConnection.update_data('questions',str(qID),newQuestion)
  return redirect(url_for('manager'))

@app.route('/manager')
def manager():
  return render_template('manager.html')

@app.route('/manager/allQuestions')
def allQuestions():
  return render_template('allQuestions.html')

@app.route('/manager/addQuestion')
def addQuestion():
  return render_template('addQuestion.html')


@app.route('/manager/addQuestion',methods = ['POST'])
def postQuestion():
  qArr = DBConnection.get_data('questions')
  key = len(qArr)
  # CRUD CREATE OPERATION
  description = request.form.get('desc')
  option1 = request.form.get('opt1')
  option2 = request.form.get('opt2')
  option3 = request.form.get('opt3')
  option4 = request.form.get('opt4')
  answer = request.form.get('corr')
  difficulty = request.form.get('diff')
  
  newQuestion = {
      'question': description,
      'option1': option1, 
      'option2': option2,
      'option3': option3,
      'option4': option4,
      'correct': answer,
      'questionlevel': difficulty    
  }
  DBConnection.update_data("questions",str(key),newQuestion)
  return redirect(url_for('manager'))



app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://235e-34-121-56-224.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [23/May/2023 12:52:21] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/May/2023 12:52:21] "GET /static/style.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/May/2023 12:52:21] "GET /static/img/image-removebg-preview.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/May/2023 12:52:21] "GET /static/img/image-removebg-preview(1).png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/May/2023 12:52:22] "GET /favicon.ico HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/May/2023 12:52:25] "GET /manager HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/May/2023 12:52:25] "GET /favicon.ico HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/May/2023 12:52:26] "GET /manager/allQuestions HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/May/2023 12:52:26] "GET /static/style.css HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [23/May/2023 12:52:26] "GET /static/img/image-removebg-preview.png HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [23/May/2023 12:52:27] "GET /favicon.

In [8]:
import backend
DBConnection = backend.Database('https://cloudclass-44ac5-default-rtdb.europe-west1.firebasedatabase.app/')
qArr = DBConnection.get_data('questions')
len(qArr)

46

In [9]:
import backend
DBConnection = backend.Database('https://cloudclass-44ac5-default-rtdb.europe-west1.firebasedatabase.app/')
db_users = DBConnection.get_data("Users")
userData = {}
for user in db_users.values():
  userData[user['username']] = user['password']

for user in db_users.values():
  print(user)

{'highscore': '10', 'id': '123', 'level': 'Medium', 'nickname': 'Racoonnnnnnn', 'password': '123', 'role': 'player', 'username': 'Rocket'}
{'highscore': '0', 'id': '1G54F', 'level': 'Easy', 'nickname': 'Star Lord', 'password': '123', 'role': 'player', 'username': 'Quill'}
{'highscore': '0', 'id': 'ffre456', 'level': 'Easy', 'nickname': 'mantissssss', 'password': '123', 'role': 'player', 'username': 'Mantis'}
{'highscore': '0', 'id': 'I6A5G90', 'level': 'Medium', 'nickname': 'IAMGROOT', 'password': '123', 'role': 'player', 'username': 'Groot'}
{'highscore': '0', 'id': 'GG1992', 'level': 'Easy', 'nickname': 'Davidik', 'password': '123', 'role': 'player', 'username': 'DavidZvonaruv'}
{'highscore': '0', 'id': '123', 'level': 'Medium', 'nickname': '123', 'password': '12', 'role': 'player', 'username': 'Quill'}


In [10]:
# from firebase import firebase
# import backend

# DBConnection = backend.Database('https://cloudclass-44ac5-default-rtdb.europe-west1.firebasedatabase.app/')
# DBConnection.update_data('questions','54',{"name":"miguel"})